In [3]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [5]:
# read in with pandas
winter1 = pd.read_csv ('hpn20-21data.txt')
winter2 = pd.read_csv ('hpn21-22data.txt')
winter3 = pd.read_csv ('hpn22-23data.txt')

In [7]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [9]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [11]:
precip_days2

,station,datetime,precip(mm),wxcodes
29,HPN,2021-12-02 05:56:00,T,-RA
30,HPN,2021-12-02 06:56:00,T,-RA
31,HPN,2021-12-02 07:56:00,0.25,-RA
32,HPN,2021-12-02 08:56:00,T,-RA BR
33,HPN,2021-12-02 09:56:00,T,-RA BR
...,...,...,...,...
2218,HPN,2022-02-25 09:56:00,0.76,-FZRA BR
2219,HPN,2022-02-25 10:56:00,1.27,-FZRA BR
2221,HPN,2022-02-25 11:56:00,0.51,-FZRA BR
2222,HPN,2022-02-25 12:12:00,1.02,-FZRAPL BR


In [13]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [15]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [17]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [19]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [21]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [25]:
hpn = LocationInfo(
    'White Plains', 'New York', 
    latitude=41.0340, 
    longitude=-73.7629, 
    timezone='America/New_York'
)

hpn

LocationInfo(name='White Plains', region='New York', timezone='America/New_York', latitude=41.034, longitude=-73.7629)

In [27]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(hpn.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [29]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(hpn.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [31]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(hpn.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [33]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:01:25  2020-12-01 16:26:59  2020-12-01
1   2020-12-02 07:02:26  2020-12-02 16:26:46  2020-12-02
2   2020-12-03 07:03:26  2020-12-03 16:26:35  2020-12-03
3   2020-12-04 07:04:24  2020-12-04 16:26:26  2020-12-04
4   2020-12-05 07:05:22  2020-12-05 16:26:19  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:38  2021-02-24 17:40:19  2021-02-24
86  2021-02-25 06:35:09  2021-02-25 17:41:30  2021-02-25
87  2021-02-26 06:33:39  2021-02-26 17:42:40  2021-02-26
88  2021-02-27 06:32:08  2021-02-27 17:43:49  2021-02-27
89  2021-02-28 06:30:36  2021-02-28 17:44:59  2021-02-28

[90 rows x 3 columns]


In [35]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:01:11  2021-12-01 16:27:02  2021-12-01
1   2021-12-02 07:02:12  2021-12-02 16:26:48  2021-12-02
2   2021-12-03 07:03:12  2021-12-03 16:26:37  2021-12-03
3   2021-12-04 07:04:10  2021-12-04 16:26:27  2021-12-04
4   2021-12-05 07:05:08  2021-12-05 16:26:20  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:37:00  2022-02-24 17:40:02  2022-02-24
86  2022-02-25 06:35:30  2022-02-25 17:41:12  2022-02-25
87  2022-02-26 06:34:00  2022-02-26 17:42:23  2022-02-26
88  2022-02-27 06:32:30  2022-02-27 17:43:32  2022-02-27
89  2022-02-28 06:30:58  2022-02-28 17:44:42  2022-02-28

[90 rows x 3 columns]


In [37]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:00:56  2022-12-01 16:27:06  2022-12-01
1   2022-12-02 07:01:57  2022-12-02 16:26:51  2022-12-02
2   2022-12-03 07:02:57  2022-12-03 16:26:39  2022-12-03
3   2022-12-04 07:03:56  2022-12-04 16:26:29  2022-12-04
4   2022-12-05 07:04:54  2022-12-05 16:26:22  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:37:21  2023-02-24 17:39:45  2023-02-24
86  2023-02-25 06:35:52  2023-02-25 17:40:55  2023-02-25
87  2023-02-26 06:34:22  2023-02-26 17:42:06  2023-02-26
88  2023-02-27 06:32:52  2023-02-27 17:43:16  2023-02-27
89  2023-02-28 06:31:20  2023-02-28 17:44:25  2023-02-28

[90 rows x 3 columns]


In [39]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [41]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2020-12-04 19:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
1,HPN,2020-12-04 20:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
2,HPN,2020-12-04 21:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
3,HPN,2020-12-04 22:56:00,0.2500,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
4,HPN,2020-12-04 23:56:00,0.5100,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
...,...,...,...,...,...,...,...
262,HPN,2021-02-27 12:56:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
263,HPN,2021-02-27 13:56:00,1.0200,-RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
264,HPN,2021-02-27 14:56:00,1.7800,RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
265,HPN,2021-02-27 15:56:00,3.0500,RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49


In [43]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2021-12-02 05:56:00,0.0001,-RA,2021-12-02,2021-12-02 07:02:12,2021-12-02 16:26:48
1,HPN,2021-12-02 06:56:00,0.0001,-RA,2021-12-02,2021-12-02 07:02:12,2021-12-02 16:26:48
2,HPN,2021-12-02 07:56:00,0.2500,-RA,2021-12-02,2021-12-02 07:02:12,2021-12-02 16:26:48
3,HPN,2021-12-02 08:56:00,0.0001,-RA BR,2021-12-02,2021-12-02 07:02:12,2021-12-02 16:26:48
4,HPN,2021-12-02 09:56:00,0.0001,-RA BR,2021-12-02,2021-12-02 07:02:12,2021-12-02 16:26:48
...,...,...,...,...,...,...,...
229,HPN,2022-02-25 08:56:00,2.7900,-SN BR,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12
230,HPN,2022-02-25 09:56:00,0.7600,-FZRA BR,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12
231,HPN,2022-02-25 10:56:00,1.2700,-FZRA BR,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12
232,HPN,2022-02-25 11:56:00,0.5100,-FZRA BR,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12


In [45]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2022-12-03 12:56:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:02:57,2022-12-03 16:26:39
1,HPN,2022-12-03 13:56:00,0.2500,-RA BR,2022-12-03,2022-12-03 07:02:57,2022-12-03 16:26:39
2,HPN,2022-12-03 14:56:00,1.2700,-RA BR,2022-12-03,2022-12-03 07:02:57,2022-12-03 16:26:39
3,HPN,2022-12-03 15:56:00,1.7800,-RA BR,2022-12-03,2022-12-03 07:02:57,2022-12-03 16:26:39
4,HPN,2022-12-03 16:56:00,3.5600,-RA BR,2022-12-03,2022-12-03 07:02:57,2022-12-03 16:26:39
...,...,...,...,...,...,...,...
201,HPN,2023-02-22 22:56:00,0.0000,-RA,2023-02-22,2023-02-22 06:40:16,2023-02-22 17:37:23
202,HPN,2023-02-23 08:56:00,0.0000,-RA BR,2023-02-23,2023-02-23 06:38:49,2023-02-23 17:38:34
203,HPN,2023-02-24 04:56:00,0.0000,-RA FG,2023-02-24,2023-02-24 06:37:21,2023-02-24 17:39:45
204,HPN,2023-02-25 15:56:00,0.0001,-SN BR,2023-02-25,2023-02-25 06:35:52,2023-02-25 17:40:55


In [47]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
10,HPN,2020-12-05 07:56:00,0.5100,-RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19
11,HPN,2020-12-05 08:56:00,1.2700,RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19
12,HPN,2020-12-05 09:56:00,2.2900,-RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19
13,HPN,2020-12-05 10:56:00,4.3200,-RA,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19
14,HPN,2020-12-05 11:56:00,5.5900,RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19
...,...,...,...,...,...,...,...
262,HPN,2021-02-27 12:56:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
263,HPN,2021-02-27 13:56:00,1.0200,-RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
264,HPN,2021-02-27 14:56:00,1.7800,RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49
265,HPN,2021-02-27 15:56:00,3.0500,RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49


In [49]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [51]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
11,HPN,2020-12-05 08:56:00,1.2700,RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19,2020-12-05 08:05:22,2020-12-05 15:26:19
12,HPN,2020-12-05 09:56:00,2.2900,-RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19,2020-12-05 08:05:22,2020-12-05 15:26:19
13,HPN,2020-12-05 10:56:00,4.3200,-RA,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19,2020-12-05 08:05:22,2020-12-05 15:26:19
14,HPN,2020-12-05 11:56:00,5.5900,RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19,2020-12-05 08:05:22,2020-12-05 15:26:19
15,HPN,2020-12-05 12:56:00,3.8100,-RA BR,2020-12-05,2020-12-05 07:05:22,2020-12-05 16:26:19,2020-12-05 08:05:22,2020-12-05 15:26:19
...,...,...,...,...,...,...,...,...,...
261,HPN,2021-02-27 11:56:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49,2021-02-27 07:32:08,2021-02-27 16:43:49
262,HPN,2021-02-27 12:56:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49,2021-02-27 07:32:08,2021-02-27 16:43:49
263,HPN,2021-02-27 13:56:00,1.0200,-RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49,2021-02-27 07:32:08,2021-02-27 16:43:49
264,HPN,2021-02-27 14:56:00,1.7800,RA FG,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49,2021-02-27 07:32:08,2021-02-27 16:43:49


In [53]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [55]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [57]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.694666037735849
2021-2022: 0.6302473684210527
2022-2023: 0.4370659574468085


In [59]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2020-12-04 19:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
1,HPN,2020-12-04 20:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
2,HPN,2020-12-04 21:56:00,0.0001,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
3,HPN,2020-12-04 22:56:00,0.2500,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
4,HPN,2020-12-04 23:56:00,0.5100,-RA,2020-12-04,2020-12-04 07:04:24,2020-12-04 16:26:26
...,...,...,...,...,...,...,...
251,HPN,2021-02-22 18:56:00,2.7900,SN FG,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
252,HPN,2021-02-22 19:56:00,3.3000,-RA BR,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
253,HPN,2021-02-22 20:56:00,1.2700,-RA BR,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
254,HPN,2021-02-22 21:56:00,1.0200,-RA BR,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57


In [61]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7448\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [63]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [65]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [67]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.2989049180327869
2021-2022: 0.4459837837837838
2022-2023: 0.5073577464788733
